In [22]:
import pyodbc
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import os
import httplib2
from googleapiclient.discovery import build
from google.oauth2 import service_account
# https://github.com/mkleehammer/pyodbc/wiki
from datetime import datetime, timezone, timedelta
import pandas as pd
from pytz import timezone
import gspread

# https://levelup.gitconnected.com/python-pandas-google-spreadsheet-476bd6a77f2b
# tz = timezone('America/Los_Angeles')
# data3 = datetime.today().astimezone().strftime("%Y-%m-%d %H:%M:%S %p %Z") # needs pytz
# data4 = datetime.today().strftime("%B %m, %Y - %I:%M %p PDT")

In [4]:
def sendEmail():
    email = os.environ.get("USER_EMAIL") # the email where you sent the email
    password = os.environ.get("USER_PW")
    send_to_email = os.environ.get("USER_EMAIL") # for whom
    message = """\
<html>
  <head></head>
  <body>
    <p>Hi!<br> <br>
       How are you?<br>
       Here is the <a href="https://docs.google.com/spreadsheets/d/1iVCGoVT6JsuRnNY7sR38e_yZxVOQZLZibmbeNGeyJgU/edit#gid=1443099774">link</a> you wanted.
       <br> <br>
       Jacob
    </p>
  </body>
</html>
"""
    msg = MIMEMultipart()
    msg["From"] = email
    msg["To"] = send_to_email
    msg["Subject"] = "Gmail prac from Python"
    msg.attach(MIMEText(message, 'html'))

    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.starttls()
    server.login(email, password)
    text = msg.as_string()
    server.sendmail(email, send_to_email, text)
    server.quit()

In [21]:
## spreadsheet stuff
try: 
    scopes = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/drive.file",
              "https://www.googleapis.com/auth/spreadsheets"]
    secret_file = os.path.join(os.getcwd(), "key/jacobs_key.json")

    spreadsheet_id = '1VxQDc-K3s7CoG245cy5deOV_cdX5CAT_HtIVpvb7doo'
    # range_name = 'current_time'
    range_name = 'current_time!A1'

    credentials = service_account.Credentials.from_service_account_file(secret_file, scopes = scopes)
    service = build('sheets', 'v4', credentials=credentials)

    values = [[datetime.today().strftime("%B %d, %Y - %I:%M %p CST")]]

    data1 = {
        "values": values
    }
    #service.spreadsheets().values() is basically always the same
    #you can do .update() or .append() after that
        # with spreadsheetId = spread
    service.spreadsheets().values().append(spreadsheetId=spreadsheet_id, body=data1, range=range_name,
                                           valueInputOption='USER_ENTERED').execute()


except OSError as e:
    print(e)

In [24]:
def get_data_google_sheets(sample_spreadsheet_id, tab_index):
    scopes = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/drive.file",
              "https://www.googleapis.com/auth/spreadsheets"]
    secret_file = os.path.join(os.getcwd(), "key/jacobs_key.json")

    # range_name = 'current_time'
    range_name = 'current_time!A1'

    credentials = service_account.Credentials.from_service_account_file(secret_file, scopes = scopes)
    
    # Request authorization and open the selected spreadsheet
    gc = gspread.authorize(credentials).open_by_key(sample_spreadsheet_id)

    # Prompts for all spreadsheet values
    values = gc.get_worksheet(tab_index).get_all_values()
    
    # Turns the return into a dataframe
    df = pd.DataFrame(values)
    df.columns = df.iloc[0]
    df.drop(df.index[0], inplace=True)         

    return df

In [29]:
df = get_data_google_sheets('1VxQDc-K3s7CoG245cy5deOV_cdX5CAT_HtIVpvb7doo', 0)
df2 = get_data_google_sheets('1VxQDc-K3s7CoG245cy5deOV_cdX5CAT_HtIVpvb7doo', 1)

In [26]:
def create_credentials():

    scopes = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/drive'
        ]
        
    credentials = service_account.Credentials.from_service_account_file(secret_file, scopes = scopes)
    
    return gspread.authorize(credentials)

In [27]:
def update_spreadsheet(id_sheet, id_spreadsheet, df):
    gc = create_credentials()
    
    gc = gc.open_by_key(id_spreadsheet)
    
    sheet = gc.get_worksheet(id_sheet)
    
    sheet.update(([df.columns.values.tolist()] + df.values.tolist()))

In [32]:
def create_insert_sheets(nm_sheet, email_share, type_role, df):
    gc = create_credentials()
    
    sh = gc.create(nm_sheet)
    sh.share(email_share, perm_type='user', role=type_role)
    
    ## 
    
    update_spreadsheet(0, sh.id, df)
    return sh

In [33]:
# service account creates an entirely new google sheet - might be useful
create_insert_sheets('newsheet', 'jyablonski9@gmail.com', 'owner', df)

<Spreadsheet 'newsheet' id:1GfSjxhb851ziT67xuOGT_6-KzElrrjQSHmeZJu4ZHt8>

In [31]:
update_spreadsheet(0, '1VxQDc-K3s7CoG245cy5deOV_cdX5CAT_HtIVpvb7doo', df)
# it will only update values in the same places, outside values stay.

In [42]:
gc = create_credentials()
# sh - gc.open('sheetsname1')
# sh - gc.open_by_key('1VxQDc-K3s7CoG245cy5deOV_cdX5CAT_HtIVpvb7doo')
sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1VxQDc-K3s7CoG245cy5deOV_cdX5CAT_HtIVpvb7doo')
worksheet1 = sh.get_worksheet(0)
worksheet2 = sh.get_worksheet(1)

In [56]:
worksheet1.append_rows(df.values.tolist())

{'spreadsheetId': '1VxQDc-K3s7CoG245cy5deOV_cdX5CAT_HtIVpvb7doo',
 'tableRange': 'current_time!A1:C10',
 'updates': {'spreadsheetId': '1VxQDc-K3s7CoG245cy5deOV_cdX5CAT_HtIVpvb7doo',
  'updatedRange': 'current_time!A11:C13',
  'updatedRows': 3,
  'updatedColumns': 3,
  'updatedCells': 9}}